# Estimations

In [1]:
import os
import pickle
import json
from pathlib import Path

In [2]:
# import cmdstanpy
# cmdstanpy.install_cmdstan()
from cmdstanpy import CmdStanModel, compile_stan_file

### Estimations

In [3]:
wd = os.getcwd()
wd_jsondata = Path(os.path.join(wd, '__jsondata__'))

In [4]:
# # delete all `pkl` files
# for file in wd_jsondata.glob("*.pkl"):
# 	file.unlink()

In [5]:
all_json_datas = list(wd_jsondata.rglob('*.json'))
len(all_json_datas)

69

In [6]:
# build stan model
stan_file = os.path.join(wd, 'real_data.stan')
output_dir = Path('./tmp')
exe_path = compile_stan_file(
	src=stan_file,
	cpp_options={
		"LDFLAGS": "-headerpad_max_install_names",
		"CXXFLAGS": "-O2",
})
model = CmdStanModel(stan_file=stan_file, exe_file=exe_path)

In [ ]:
contest_max_longivity = 800

# fit the model with data
for json_data in all_json_datas:
	for file in output_dir.iterdir():
		if file.is_file():
			file.unlink()
	contest_name = json_data.stem
	posterior_file = f'{contest_name}.pkl'
	posterior_file_path = wd_jsondata.joinpath(posterior_file)
	if posterior_file_path.exists():
		continue
	with open(json_data, "r", encoding="utf-8") as f:
		data_json = json.load(f)
	if data_json['N_Delta'] > contest_max_longivity:
		continue
	print(contest_name, 'begin')
	fit = model.sample( \
		data=json_data,
		iter_warmup=1000,
		iter_sampling=2000,
		chains=4,
		parallel_chains=4,
		show_console=False,
		max_treedepth=12,  # for difficult model
		adapt_delta=0.99,  # for difficult model
		output_dir=output_dir,
		seed=12345,
	)
	posteriors = fit.stan_variables()
	with open(posterior_file_path, 'wb') as f:
		pickle.dump(posteriors, f)
	print(contest_name, 'end')

21:35:34 - cmdstanpy - INFO - CmdStan start processing


contest_6322 begin


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status